In [ ]:
import pandas as pd

path = r"C:\Users\mg\Desktop\Phyton\2024.xlsx"


xls = pd.ExcelFile(path)
print("Sheets:", xls.sheet_names)

df = pd.read_excel(path, sheet_name=0)


df.info()
df.head()
df.columns.tolist()

na_report = df.isna().sum().sort_values(ascending=False)
na_report.head(15)


Sheets: [' Sheet 1']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Columns: 104 entries, ND (Declaration Number) to FIRM (Additional information about the contract holder (Rosstat))
dtypes: float64(31), int64(18), object(55)
memory usage: 273.9+ KB


G31_71 (Name of additional unit)                       337
G41A (Additional unit code)                            337
?                                                      335
G31_81 (Name of the second unit of measurement)        319
G31_82 (Second unit code)                              319
G07 (GTE type)                                         312
G30 (Station/Warehouse Name)                           244
G027 (Sender's checkpoint)                             214
G020 (OGRN of the sender)                              212
G021 (Sender's Taxpayer Identification Number)         212
G024B (Sender's OKATO code)                            211
G301 (Certificate of Temporary Storage Warehouse)      208
G541 (Broker Certificate Number)                       199
G541_NAM (Broker Name)                                 199
G541_ADR (Broker's address and contact information)    199
dtype: int64

In [8]:
df = df.rename(columns={
    "G33 (Commodity code according to TN VED)": "HS_code",
    "G31_13 (Country of origin)": "origin_country",
    "G17B (Destination country)": "dest_country",
    "G38 (Net weight, kg)": "net_kg",
    "USDKG (USD per KG)": "usd_per_kg",
    "ND (Declaration Number)": "decl_id",
    "G072 (Registration Date)": "reg_date",
    "G07 (GTE type)": "goods_type",
    "G011 (Direction of movement)": "Export/Import",
    "G022 (Sender's name)": "Sender",
    "G082 (Name of recipient)": "Recipient",
    "G202 (Delivery Condition)": "Incoterms",
    "G42RUB (Invoice value in rubles)": "Price_rub",


    

})


In [9]:
df["reg_date"] = pd.to_datetime(df["reg_date"], errors="coerce", dayfirst=True)

In [10]:
df["reg_date"].head()


0   2024-02-18
1   2024-11-18
2   2024-03-07
3   2024-12-03
4   2024-12-03
Name: reg_date, dtype: datetime64[ns]

In [13]:
import pandas as pd

# 1. Data and Month
df["reg_date"] = pd.to_datetime(df["reg_date"], errors="coerce", dayfirst=True)
df["month"] = df["reg_date"].dt.to_period("M")

# 2. Value in usd (if needed)
df["value_usd"] = df["net_kg"] * df["usd_per_kg"]

# 3. Grouping by month + Import/Export
monthly_summary = (
    df.groupby(["month", "Export/Import"])
      .agg(
          total_kg=("net_kg", "sum"),
          total_value_usd=("value_usd", "sum")
      )
      .unstack("Export/Import")
      .fillna(0)
)

# 4. column names
monthly_summary.columns = [
    f"{metric}_{direction}"
    for metric, direction in monthly_summary.columns
]

monthly_summary = monthly_summary.reset_index()

# 5. Adding TOTAL row
total_row = monthly_summary.select_dtypes("number").sum()
total_row["month"] = "TOTAL"

monthly_summary_with_total = pd.concat(
    [monthly_summary, total_row.to_frame().T],
    ignore_index=True
)

monthly_summary_with_total

,month,total_kg_Export,total_kg_Import,total_value_usd_Export,total_value_usd_Import
0,2024-01,65429.56,191353.89,113630.6548,905064.2496
1,2024-02,53424.416,62393.58,100623.93326,222645.499
2,2024-03,120613.11,358793.042,227202.451,1516166.38234
3,2024-04,172389.528,186505.19,397303.66496,402549.3735
4,2024-05,269703.21,115018.831,322596.3259,463390.3324
5,2024-06,86633.76,130365.37,164715.3219,391984.822
6,2024-07,30254.485,77635.3,107484.17205,189540.112
7,2024-08,76287.668,53628.46,184804.7962,182545.77902
8,2024-09,88375.0,50146.94,181769.18,207093.23839
9,2024-10,74601.6,81709.264,210769.974,215765.5412


In [88]:
top_export_import = (
    df.groupby("Export/Import")["net_kg"]
      .sum()
      .sort_values(ascending=False)
      .head(100)
)
top_export_import

# convert Series to DataFrame so we can append a total row
top_export_import_df = top_export_import.to_frame()

total_row = pd.DataFrame([top_export_import_df.sum(numeric_only=True)], index=["TOTAL"])

top_export_import_with_total = pd.concat([top_export_import_df, total_row])

top_export_import_with_total

,net_kg
Export,1376147.110
Import,1361539.637
TOTAL,2737686.747


In [14]:
top_tnved = (
    df.groupby(["HS_code", "Export/Import"])["net_kg"]
      .sum()
      .unstack(fill_value=0)
)

# take top 10 HS_code by total net_kg (sum across Export/Import)
top_tnved = (
    top_tnved.assign(total=top_tnved.sum(axis=1))
             .sort_values("total", ascending=False)
             .head(10)
             .drop(columns="total")
)
top_tnved
total_row = pd.DataFrame([top_tnved.sum(numeric_only=True)], index=["TOTAL"])
top_tnved_with_total = pd.concat([top_tnved, total_row])

top_tnved_with_total

Export/Import,Export,Import
3105100000,397750.15,1164384.957
3101000000,978396.96,197154.680
TOTAL,1376147.11,1361539.637


In [ ]:
top_sender_countries_import = (
    df[df["Export/Import"] == "Import"]
      .groupby("origin_country")["net_kg"]
      .sum()
      .sort_values(ascending=False)
      .head(10)
)
top_sender_countries_import
def new_func(top_sender_countries_import):
    top_sender_countries_import_with_total = pd.concat([
    top_sender_countries_import,
    pd.Series({"TOTAL": top_sender_countries_import.sum()})
])

    top_sender_countries_import_with_total

new_func(top_sender_countries_import)

CHINA           336077.810
SPAIN           271083.670
ITALY           207499.050
Türkiye         179408.920
NETHERLANDS     126887.500
SERBIA          125239.970
BELGIUM          96056.800
RUSSIA            7665.280
GERMANY           5621.747
PORTUGAL          5000.000
TOTAL          1360540.747
dtype: float64

In [18]:
top_sender_senders_import = (
    df[df["Export/Import"] == "Import"]
      .groupby("Sender")["net_kg"]
      .sum()
      .sort_values(ascending=False)
      .head(10)
)
top_sender_senders_import

top_sender_senders_import_with_total = pd.concat([
    top_sender_senders_import,
    pd.Series({"TOTAL": top_sender_senders_import.sum()})
])

top_sender_senders_import_with_total

QINGDAO SEAWIN BIOTECH GROUP CO.LTD.                                                                               263272.00
AVAGRO TARIM KIMYA URETIM ITH.IHR.PAZ.SAN. VE TIC. LTD.                                                            160169.88
SYNGENTA CROP PROTECTION AG.                                                                                       139647.00
FERTICO DOO                                                                                                        125239.97
MCC ENERJI URETIM TURIZMINSAAT EMLAK SAN. VE TICARET ANONIM SIRKETI."                                              124402.00
LIMA EUROPE NV.                                                                                                     96054.80
BEIJING LEILI MARINE BIOINDUSTRY INC..                                                                              69428.00
CHAMPFOOD INTERNATIONAL.                                                                                            63000.00


In [21]:
top_recipient_recipient_import = (
    df[df["Export/Import"] == "Import"]
      .groupby("Recipient")["net_kg"]
      .sum()
      .sort_values(ascending=False)
      .head(10)
)
top_recipient_recipient_import

top_recipient_recipient_import_with_total = pd.concat([
    top_recipient_recipient_import,
    pd.Series({"TOTAL": top_recipient_recipient_import.sum()})
])

top_recipient_recipient_import_with_total

OOO "ECO KULTURA RUS"           462498.00
OOO "SINGENTA"                  180822.60
OOO "AVAGRO RUS"                160169.88
LLC "YUGPOLIV KOROLEV AGRO"     125130.75
LLC "GRANUM"                     69428.00
LLC "MASTER CHAMP"               63000.00
LLC "GROUTEK"                    56890.00
LLC "FLORA GROW COMPANY"         44317.28
LLC "SINGENTA"                   26676.00
NUTRITECH RUS LLC                25850.06
TOTAL                          1214782.57
dtype: float64

In [22]:
top_sender_countries_export = (
    df[df["Export/Import"] == "Export"]
      .groupby("origin_country")["net_kg"]
      .sum()
      .sort_values(ascending=False)
      .head(10)
)
top_sender_countries_export

top_sender_countries_export_with_total = pd.concat([
    top_sender_countries_export,
    pd.Series({"TOTAL": top_sender_countries_export.sum()})
])

top_sender_countries_export_with_total

RUSSIA     1354117.61
Türkiye      21700.00
ITALY          329.50
TOTAL      1376147.11
dtype: float64

In [23]:
top_sender_senders_export = (
    df[df["Export/Import"] == "Export"]
      .groupby("Sender")["net_kg"]
      .sum()
      .sort_values(ascending=False)
      .head(10)
)
top_sender_senders_export

top_sender_senders_export_with_total = pd.concat([
    top_sender_senders_export,
    pd.Series({"TOTAL": top_sender_senders_export.sum()})
])

top_sender_senders_export_with_total

LLC "GREEN-MS"                               354700.0
INTERKROS CENTER LLC                         123618.0
LLC NPO <ALFA-GROUP>                          85231.0
LLC "AGROTECH GUMAT"                          82065.0
JSC "VIKTAN"                                  68220.0
JOINT-STOCK COMPANY "CHICKEN KINGDOM"         62000.0
LIMITED LIABILITY COMPANY "EDAGUM SM RUS"     57037.0
LLC NVP "BASHINKOM"                           46458.0
LLC "NVP "BASHINKOM"                          38960.0
IP CHUNOSOV SERGEY NIKOLAEVICH                38520.0
TOTAL                                        956809.0
dtype: float64

In [27]:
top_recipient_recipient_export = (
    df[df["Export/Import"] == "Export"]
      .groupby("Recipient")["net_kg"]
      .sum()
      .sort_values(ascending=False)
      .head(10)
)
top_recipient_recipient_export

top_recipient_recipient_export_with_total = pd.concat([
    top_recipient_recipient_export,
    pd.Series({"TOTAL": top_recipient_recipient_export.sum()})
])

top_recipient_recipient_export_with_total

ABALIOGLU LEZITA GIDA SAN AS.                                         189010.0
ABALIOGLU LEZITA GUIDA SAN AS                                         165690.0
ALARID ALZARQA FOR ORGANIC FERTILIZER & PLANT FEED TRADING CO. LLC    123516.0
IFFCO-MC CROP SCIENCE PRIVATE LIMITED.                                 63000.0
GENESIS FERTILIZERS GLOBAL SOLUTIONS (PVT) LTD.                        59976.0
LLC "TECHNIKHAUS"                                                      57780.0
AN PHONG CT PRODUCTION TRADING COMPANY LIMITED.                        40000.0
LLC "TAMINOTI SARBAND"                                                 39197.0
IP DURDYEV GURBAN                                                      38956.0
GOLDEN DRAGON GLOBAL IND. GROUP COMPANY LIMITED.                       38715.0
TOTAL                                                                 815840.0
dtype: float64

In [ ]:
# 0. incoterms
incoterms_pivot = (
    df.groupby(["Incoterms", "Export/Import"])
      .agg(
          total_kg=("net_kg", "sum"),
          total_value_usd=("value_usd", "sum"),
          avg_usd_per_kg=("usd_per_kg", "mean")
      )
      .unstack("Export/Import")
      .fillna(0)
)

# 1. rename columns
incoterms_pivot.columns = [
    f"{metric}_{direction}"
    for metric, direction in incoterms_pivot.columns
]

# 2. return index to column
incoterms_pivot = incoterms_pivot.reset_index()

# 3. total row
total_row = incoterms_pivot.select_dtypes("number").sum()
total_row["Incoterms"] = "TOTAL"

# 4. assemble table with total
incoterms_pivot_with_total = pd.concat(
    [incoterms_pivot, total_row.to_frame().T],
    ignore_index=True
)

incoterms_pivot_with_total




,Incoterms,total_kg_Export,total_kg_Import,total_value_usd_Export,total_value_usd_Import,avg_usd_per_kg_Export,avg_usd_per_kg_Import
0,CFR,29386.0,315841.48,82401.74,520354.51,4.6725,7.301613
1,CIF,694459.0,338096.0,1001516.5,932136.88,1.672105,2.769412
2,CIP,201014.0,207498.6,391413.3,1360315.882,1.91,6.546818
3,CPT,38698.21,23308.055,110028.5099,638259.92075,6.94,271.267091
4,DAF,2981.5,0.0,3160.39,0.0,1.06,0.0
5,DAP,32190.26,14574.93,106770.3598,178093.9703,6.768571,65.207692
6,DAT,539.0,165.0,7114.8,867.9,13.2,5.26
7,EXW,39734.56,49780.171,48180.1488,287958.1899,1.545,22.883333
8,FCA,336695.08,335745.759,498228.76391,1246306.75231,5.807321,152.473276
9,FOB,0.0,76528.842,0.0,281970.96794,0.0,3.66
